## Сентиментный анализ

#### Импортируем все библиотеки: 

In [3]:
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier as XGBoostClassifier

import nltk
import re
import string

#### Загрузим топ-слова:

In [8]:
stop_words = nltk.corpus.stopwords.words('english')

#### Теперь получим данные:

In [9]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None, names=['text', 'target'])
test = pd.read_csv('products_sentiment_test.tsv', sep='\t', index_col='Id')
sample = pd.read_csv('products_sentiment_sample_submission.csv')

In [10]:
train.text.head()

0            2 . take around 10,000 640x480 pictures .
1    i downloaded a trial version of computer assoc...
2    the wrt54g plus the hga7t is a perfect solutio...
3    i dont especially like how music files are uns...
4    i was using the cheapie pail ... and it worked...
Name: text, dtype: object

#### Функция, которая оценивает классификатор:

In [11]:
def score(clf):
    scores = cross_val_score(clf, train.text, train.target, cv=5)
    print("Score mean CLF = {:.5f}, std = {:.5f}".format(scores.mean(), scores.std()))

In [6]:
clf = make_pipeline(CountVectorizer(), MultinomialNB())
score(clf)

Score mean CLF = 0.77898, std = 0.01908


#### Оценим сразу несколько классификторов из бибилиотеки sklearn:

In [7]:
for classf in [LogisticRegression, SGDClassifier, LinearSVC, MultinomialNB, RandomForestClassifier, XGBoostClassifier]:
    clf = make_pipeline(TfidfVectorizer(), classf())
    score(clf)

Score mean CLF = 0.76650, std = 0.01107
Score mean CLF = 0.75400, std = 0.01150


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/s

Score mean CLF = 0.76849, std = 0.01442
Score mean CLF = 0.70700, std = 0.00717
Score mean CLF = 0.71101, std = 0.01257
Score mean CLF = 0.71601, std = 0.01021


#### Теперь попробуем заменить TfidfVectorizer на CountVectorizer и заново оценить результаты: 

In [8]:
for classf in [LogisticRegression, SGDClassifier, LinearSVC, MultinomialNB, RandomForestClassifier, XGBoostClassifier]:
    clf = make_pipeline(CountVectorizer(), classf())
    score(clf)

Score mean CLF = 0.76850, std = 0.00763
Score mean CLF = 0.74200, std = 0.00698


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/s

Score mean CLF = 0.75400, std = 0.01364
Score mean CLF = 0.77898, std = 0.01908
Score mean CLF = 0.72904, std = 0.02307
Score mean CLF = 0.73202, std = 0.01557


#### Лучший результат мы получили при использовании наивного байесовского классификатора, MultinomialNB

#### Теперь обучим классификатор:

In [15]:
clf = make_pipeline(CountVectorizer(ngram_range=(1,1)), MultinomialNB())
clf.fit(train.text, train.target)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

#### Сделаем предсказания и сохраним их предсказания:

In [10]:
predictions = clf.predict(test.text)

In [11]:
out = pd.DataFrame(predictions, index=test.index, columns=['y'])

In [12]:
out.to_csv('submission.csv')

#### Кажется что наш результат можно улучшить, пробуем библиотеку nltk.

Определим функцию, которая представляет слова из предложения в словарь:

In [13]:
import nltk
 
def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})
print(format_sentence("The cat is very cute"))

{'cute': True, 'The': True, 'is': True, 'cat': True, 'very': True}


Подготовим данные: 

In [14]:
pos = []
neg = []

In [15]:
for i in range(len(train)):
    if train.target[i] == 1:
        pos.append([format_sentence(train.text[i]), 'pos'])

In [16]:
for i in range(len(train)):
    if train.target[i] == 0:
        neg.append([format_sentence(train.text[i]), 'neg'])

In [17]:
training = pos[:int((.8)*len(pos))] + neg[:int((.8)*len(neg))]
testing = pos[int((.8)*len(pos)):] + neg[int((.8)*len(neg)):]

Возьмем наивный байесовский классификатор и обучим его: 

In [18]:
from nltk.classify import NaiveBayesClassifier
 
classifier = NaiveBayesClassifier.train(training)

In [19]:
classifier.show_most_informative_features()

Most Informative Features
                    love = True              pos : neg    =     15.8 : 1.0
                annoying = True              neg : pos    =      9.9 : 1.0
              viewfinder = True              neg : pos    =      9.9 : 1.0
                    disc = True              neg : pos    =      9.9 : 1.0
                    kept = True              neg : pos    =      8.8 : 1.0
                       % = True              neg : pos    =      8.8 : 1.0
                    gave = True              neg : pos    =      8.8 : 1.0
                   great = True              pos : neg    =      8.2 : 1.0
                      wo = True              neg : pos    =      7.6 : 1.0
                customer = True              neg : pos    =      7.4 : 1.0


Проверим работу классификатора:

In [20]:
example1 = "Cats are awesome!"
 
print(classifier.classify(format_sentence(example1)))

pos


Оценим качество:

In [21]:
from nltk.classify.util import accuracy
print(accuracy(classifier, testing))

0.743142144638404


#### Результат оказался даже хуже предыдущего.

#### Попробуем новую модель от OpenAI, sentiment neyron. https://arxiv.org/abs/1704.01444

Загрузим модель, предварительно склонировав ее: https://github.com/openai/generating-reviews-discovering-sentiment

In [1]:
from encoder import Model
model=Model()

Представим наш текст в векторном пространстве: 

In [23]:
text_features = model.transform(test.text)

Теперь попробуем модель на нашем тестовом тексте и запигем результаты в файл:

In [24]:
f = open('submisssion_1.csv', 'a')
f.write('Id,y' + '\n')

for i in range(len(test.text)):
    res = 0 if text_features[i, 2388] <=0 else 1
    f.write(str(i)+','+str(res) + '\n')
f.close()

### Загрузим результаты на Kaggle и получим результат 0.90749, 
### что соответствует 6 месту на Leaderboard. Vladimir Sveshnikov.

Теперь для визуализации нашей модели, нам необходимо сохранить объект модели, используя Pickle.